In [ ]:
import typing as t

import numpy as np 
import matplotlib.pyplot as plt 
from scipy.ndimage.filters import convolve

# add src to path
sys.path.append('../..')
from utils.db_helper import get_image_data
from utils.color_space_operations import get_difference_img_gen, calculate_difference_image

In [ ]:
GRAYSCALE=False
base_save_path=base_save_path = os.path.join("..", "..", "..", "exp", "Histogram")


In [ ]:
SOURCE_DIR_STYLE = R"F:\master-thesis-databases\classification_db\fake"
SOURCE_DIR_FFHQ = R"F:\master-thesis-databases\classification_db\real"

In [ ]:
def hist_peek_point(np_img: np.ndarray, bins=256, hist_range=(0,255), channels=9)-> t.Tuple[float, int]:
    peek_points = []
    for colorspace in range(channels):
        np_hist, bins = np.histogram(np_img[:,:,colorspace], density=True, bins=bins, range=hist_range)
        y = np_hist.max()
        idx = np.argwhere(np_hist==y)
        if len(idx)>1:
            idx = int(idx[0])
        else:
            idx = int(idx)
            
        x = int(bins[idx])
        peek_points.append((x,y))
    return peek_points

In [ ]:
def peek_points_from_diff_dataset(dataset_gen: t.Generator[np.ndarray, None, None], bins=511, max_number_of_images=10, grayscale=False)-> t.Tuple[t.List[float], t.List[int]]:
    print("Processing database...")
    peek_points, peek_points_y = [], []
    for np_img in dataset_gen:
        x = hist_peek_point(np_img)
        peek_points.append(x)
    return peek_points

# Peek points

In [ ]:
color_components = ("R", "G", "B", "H", "S", "V", "Y", "Cb", "Cr")
categories = ['real', 'fake']
GRAYSCALE=False

In [ ]:
ffhq_gen = get_image_data(SOURCE_DIR_FFHQ, type='float', grayscale=False, max_number_of_images=100)
style_gen = get_image_data(SOURCE_DIR_STYLE, type='float', grayscale=False, max_number_of_images=100)

FFHQ_peek_points = peek_points_from_diff_dataset(get_difference_img_gen(ffhq_gen), max_number_of_images=100, grayscale=GRAYSCALE)
Style_peek_points = peek_points_from_diff_dataset(get_difference_img_gen(style_gen), max_number_of_images=100, grayscale=GRAYSCALE)

In [ ]:
np_FFHQ_peek_points = np.asarray(FFHQ_peek_points)
np_Style_peek_points = np.asarray(Style_peek_points)

In [ ]:
for colorspace in range(len(color_components)):
    fig = plt.figure(figsize=(15,15))
    ax1 = fig.add_subplot(111)

    ax1.scatter(np_FFHQ_peek_points[:,colorspace,0], np_FFHQ_peek_points[:,colorspace,1], color='blue', label=categories[0])
    ax1.scatter(np_Style_peek_points[:,colorspace,0], np_Style_peek_points[:,colorspace,1], color='orange', label=categories[1])
    fig.suptitle(f'Diff histogram peek points comparision for color: {color_components[colorspace]}')
    plt.legend(loc='upper left')
    plt.show()
#fig.savefig('../../../exp/Histograms/Peek_points_comparision_v2.png')

# Histograms (better version in color components)

In [ ]:
def avg_hist_from_diff_img(src_path: str, max_number_of_images=10, bins=511, grayscale=False)-> t.Tuple[np.ndarray, range]:
    print("Loading dataset...")
    hist_range = (-255,256)  #ToDo automatic generation based on bins
    dataset_gen = get_image_data(src_path, type='int', grayscale=grayscale)
    print("Processing database...")
    number_of_images = 0
    np_avg_hist = np.zeros((bins,), dtype=np.float64)
    for img in dataset_gen:
        diff_img = calculate_difference_image(img)
        hist, _ = np.histogram(diff_img, density=True, bins=bins, range=hist_range)
        np_avg_hist += hist
        number_of_images += 1
        if np_avg_hist.max() > 0.95*np.finfo(np_avg_hist.dtype).max:
            raise ValueError("Dataset to large")
            break
        elif number_of_images == max_number_of_images:
            print("Finish calcluating avg histogram")
            break
    return np_avg_hist / number_of_images, range(hist_range[0], hist_range[1])


In [ ]:
np_ffhq_avg_hist, bins = avg_hist_from_diff_img(SOURCE_DIR_FFHQ, max_number_of_images=100, grayscale=GRAYSCALE)
np_style1_avg_hist, bins = avg_hist_from_diff_img(SOURCE_DIR_STYLE, max_number_of_images=100, grayscale=GRAYSCALE)

In [ ]:
fig, axs = plt.subplots(ncols=2, sharey=True, figsize=(15,7))
fig.suptitle('Comparision of average histograms from difference images')
axs[0].plot(list(bins), np_ffhq_avg_hist)
axs[0].set_title('FFHQ')
axs[1].plot(list(bins), np_style1_avg_hist, color="orange")
axs[1].set_title('StyleGan')
#fig.savefig('../../../exp/Histograms/Avg_comparision.png')